In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.input_area pre {font-family: Consolas; font-size: 14pt; line-height: 140%;}</style>"))
display(HTML("<style>.output_area pre {font-family: Consolas; font-size: 14pt; line-height: 140%;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import os
import glob
import datetime
import shutil
from collections import defaultdict, Counter
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family=font_name)

# LINCS와 GDSC에서 겹치는 compound의 expression 정보 가져오기

In [13]:
## BRD id list of the common compounds between LINCS and NCI&CGP
# cid to LSM mapping
LSM2cid_df=pd.read_table('./data/LINCS/LSM2Cid.txt', sep='\t',engine='python')
LSM2cid_df=LSM2cid_df[~LSM2cid_df['SM_PubChem_CID'].isna()]
LSM2cid_df['SM_PubChem_CID']=LSM2cid_df['SM_PubChem_CID'].astype(int)
LSM2cid_df.set_index('SM_PubChem_CID', inplace=True)
cid2LSM_ds=LSM2cid_df['SM_LINCS_ID']
cid2LSM=cid2LSM_ds.to_dict()

# LSM to BRD mapping
LSM2BRD_df=pd.read_table('./data/LINCS/BRD2LSM.txt',index_col='SM_LINCS_ID', sep='\t',engine='python')
LSM2BRD_ds=LSM2BRD_df['SM_Center_Canonical_ID']
LSM2BRD_ds=LSM2BRD_ds[LSM2BRD_ds.str.startswith('BRD')]
LSM2BRD=LSM2BRD_ds.to_dict()

def get_LINCS_BRD_list(cell):
    ic50_df = pd.read_table('./result/GDSC_IC50s_for_common_drugs.txt', sep='\t',index_col=0, engine='python')
    ic50_ds=ic50_df[cell]
    ic50_ds=ic50_ds[~ic50_ds.isna()]
    comp_list=[]
    for cid in list(ic50_ds.index):
        comp_list.append(LSM2BRD[cid2LSM[cid]])
    comp_list.append('DMSO')
    return comp_list

# main
BRD_dic={}
for cell in ['A375','A549','BT20','HS578T','HT29','JURKAT','LNCAP','MCF7','MDAMB231','PC3']:
    BRD_list=get_LINCS_BRD_list(cell)
    BRD_dic[cell]=BRD_list

BRD_dic

{'A375': ['BRD-K97764662',
  'BRD-K68407802',
  'BRD-K81418486',
  'BRD-K64052750',
  'BRD-K74065929',
  'BRD-K59184148',
  'BRD-K79404599',
  'BRD-K81528515',
  'BRD-K25311561',
  'BRD-K99964838',
  'BRD-K51313569',
  'BRD-K08799216',
  'BRD-K29395450',
  'BRD-K05104363',
  'BRD-K17743125',
  'BRD-K56751279',
  'BRD-K13566078',
  'BRD-K49865102',
  'BRD-K25340465',
  'BRD-K76908866',
  'BRD-K67868012',
  'BRD-K51575138',
  'BRD-K53414658',
  'BRD-K72703948',
  'BRD-K88560311',
  'BRD-K52751261',
  'BRD-K16189898',
  'BRD-K48488978',
  'BRD-K71035033',
  'BRD-K47983010',
  'BRD-A60245366',
  'BRD-K74514084',
  'BRD-K57080016',
  'BRD-U86686840',
  'BRD-K66538826',
  'BRD-K00337317',
  'BRD-K13390322',
  'BRD-A10523515',
  'BRD-K99749624',
  'BRD-K60866521',
  'BRD-K07667918',
  'BRD-K63068307',
  'BRD-K99616396',
  'BRD-K12343256',
  'BRD-K99545815',
  'BRD-K49669041',
  'BRD-K87142802',
  'BRD-K12184916',
  'BRD-K12502280',
  'BRD-K79259477',
  'BRD-K52911425',
  'BRD-K02113016',
  'B

In [14]:
## meta-data of LINCS experiments
# main
lns_file = open('F:/LINCS_data/Broad_LINCS_Level3_INF_mlr12k_n113012x22268_2015-12-31.gct')

lineCnt = 0
data_dList = []
for line in lns_file:
    lineCnt += 1
    if lineCnt <= 2: continue
    if lineCnt == 16: break

    line_list = line.replace('\n','').replace('\r','').split('\t')
    data_dList.append(line_list)

lns_file.close()


lns_df = pd.DataFrame(data_dList[1:],columns=data_dList[0]).set_index('id')
lns_df.iloc[:,:20]

,pr_analyte_id,pr_analyte_num,pr_bset_id,pr_gene_id,pr_gene_symbol,pr_gene_title,pr_is_bing,pr_is_inf,pr_is_lmark,pr_lua_id,pr_model_id,pr_pool_id,A375-ctl_vehicle-1,A375-ctl_vehicle-101796,A375-ctl_vehicle-101797,A375-ctl_vehicle-102172,A375-ctl_vehicle-102173,A375-ctl_vehicle-1071,A375-ctl_vehicle-1072,A375-ctl_vehicle-1073
id,,,,,,,,,,,,,,,,,,,,
cl_center_specific_id,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,A375,A375,A375,A375,A375,A375,A375,A375
rn_target_gene_id,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666
sm_center_compound_id,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,dmso,DMSO,DMSO,DMSO,DMSO,dmso,dmso,dmso
sm_dose,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666
sm_dose_unit,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666
sm_lincs_id,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,DMSO,DMSO,DMSO,DMSO,DMSO,DMSO,DMSO,DMSO
sm_name,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,DMSO,DMSO,DMSO,dmso,dmso,DMSO,DMSO,DMSO
sm_pert_type,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,ctl_vehicle,ctl_vehicle,ctl_vehicle,ctl_vehicle,ctl_vehicle,ctl_vehicle,ctl_vehicle,ctl_vehicle
sm_time,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,-666,24,6,6,6,6,24,24,24


In [15]:
# LINCS experiments columns of the common compounds for each of the three cell lines
# main
lns_compExp_dic={}
for cell in BRD_dic.keys():
    lns_compExp_list=[]
    for col in list(lns_df):
        lns_cell = lns_df.loc['cl_center_specific_id', col]
        if lns_cell!=cell:
            continue
            
        hr = lns_df.loc['sm_time', col]
        if hr!='24':
            continue
            
        comp = lns_df.loc['sm_lincs_id', col]
        if comp.upper() == 'DMSO': comp = 'DMSO'
        if comp not in BRD_dic[cell]:
            continue
        
        lns_compExp_list.append(col)
            
    lns_compExp_dic[cell]=lns_compExp_list

lns_compExp_dic

{'A375': ['A375-ctl_vehicle-1',
  'A375-ctl_vehicle-1071',
  'A375-ctl_vehicle-1072',
  'A375-ctl_vehicle-1073',
  'A375-ctl_vehicle-1074',
  'A375-ctl_vehicle-1075',
  'A375-ctl_vehicle-1076',
  'A375-ctl_vehicle-122',
  'A375-ctl_vehicle-123',
  'A375-ctl_vehicle-124',
  'A375-ctl_vehicle-125',
  'A375-ctl_vehicle-126',
  'A375-ctl_vehicle-127',
  'A375-ctl_vehicle-2',
  'A375-ctl_vehicle-22',
  'A375-ctl_vehicle-22280',
  'A375-ctl_vehicle-22281',
  'A375-ctl_vehicle-22282',
  'A375-ctl_vehicle-22283',
  'A375-ctl_vehicle-223',
  'A375-ctl_vehicle-22302',
  'A375-ctl_vehicle-22303',
  'A375-ctl_vehicle-22304',
  'A375-ctl_vehicle-22305',
  'A375-ctl_vehicle-224',
  'A375-ctl_vehicle-22400',
  'A375-ctl_vehicle-22401',
  'A375-ctl_vehicle-22402',
  'A375-ctl_vehicle-22403',
  'A375-ctl_vehicle-22404',
  'A375-ctl_vehicle-22405',
  'A375-ctl_vehicle-22497',
  'A375-ctl_vehicle-22498',
  'A375-ctl_vehicle-22499',
  'A375-ctl_vehicle-225',
  'A375-ctl_vehicle-22500',
  'A375-ctl_vehicle

In [17]:
# a subset of the whole LINCS expression data(~40G) covering the common compounds for each cell
def getColIndex(line_list, colName_list):
    colIndex_list = []
    
    for colName in colName_list:
        try:
            colIndex_list.append(line_list.index(colName))
        except:
            print(colName)
            raise ValueError ('No exist column names')
            
    return colIndex_list

def getCols(fileName, colNameLine, colName_list):
    in_file = open(fileName)
    
    col_dList = []
    
    lineCnt = 0
    data_flg = False
    for line in in_file:
        lineCnt += 1
        if lineCnt%1000==0: print(lineCnt)
        line_list = line.replace('\n','').split('\t')
        
        if lineCnt == colNameLine:
            colIndex_list = getColIndex(line_list, colName_list)
            data_flg = True
            
        if data_flg:
            col_dList.append([line_list[i] for i in colIndex_list])
        
    in_file.close()
    
    return col_dList

# main
for cell, lns_compExp_list in lns_compExp_dic.items():
    print(cell)
    dList = getCols('F:/LINCS_data/Broad_LINCS_Level3_INF_mlr12k_n113012x22268_2015-12-31.gct',3,['id','pr_gene_id','pr_gene_symbol'] + lns_compExp_list)

    lnsFil_file = open('result/LINCS_expression/LINCS_expression_{}.txt'.format(cell),'w+')
    for sList in dList:
        _dum=lnsFil_file.write('\t'.join(sList)+'\n')

    lnsFil_file.close()

A375
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
A549
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
BT20
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
HS578T
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
HT29
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
JURKAT
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
LNCAP
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
MCF7
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
1

# LINCS와 CGP&NCI에서 겹치는 compound의 fold-change 구하기

In [18]:
def quantileNormalize(df_input):
#     #another algorithm
#     rank_mean = subLns_df.stack().groupby(subLns_df.rank(method='first').stack().astype(int)).mean()
#     qtSubLns_df = subLns_df.rank(method='min').stack().astype(int).map(rank_mean).unstack()

    df = df_input.copy()
    #compute rank
    dic = {}
    for col in df:
        dic.update({col : sorted(df[col])})
    sorted_df = pd.DataFrame(dic)
    rank = sorted_df.mean(axis = 1).tolist()
    #sort
    for col in df:
        t = np.searchsorted(np.sort(df[col]), df[col])
        df[col] = [rank[i] for i in t]
    return df

def getDirectProbe():
    probe_file = open('data/LINCS/probe_directed_GPL20573.txt')

    directProbe_list = []
    for line in probe_file:
        probe = line.strip()
        directProbe_list.append(probe)
        
    probe_file.close()
    return directProbe_list

## fold changes of both all and landmark genes from the experiments of the common compounds
## main
for cell in lns_compExp_dic.keys():
    in_file='result/LINCS_expression/LINCS_expression_{}.txt'.format(cell)
    for mode in ['all','lmk']:
        print(cell, mode)
        # read expression data in LINCS per cell
        lns_df = pd.read_table(in_file, sep='\t', index_col = 'id', dtype = 'str')

        # get distinct plate name
        pt_list = [pt for pt in list(set(lns_df.loc['det_plate'])) if pt != '-666']

        all_fold_change_df=pd.DataFrame()
        for pt in pt_list:
            # get expression and head data
            lnsDatHead_df = lns_df.loc[:,lns_df.loc['det_plate']==pt]

            # get only head data
            lnsHead_df = lnsDatHead_df.loc[['sm_lincs_id','sm_dose']]

            # get only expression data
            lnsDat_df = lnsDatHead_df.iloc[12:]
            # convert expression data type to numeric
            lnsDat_df = lnsDat_df.astype(float)
            # quantileNormalization
            lnsDat_df = quantileNormalize(lnsDat_df)

            # attach gene info
            lnsG_df = lns_df.iloc[12:,[1]]
            lnsGdat_df = pd.concat([lnsG_df, lnsDat_df], axis=1)
            # get expression average per gene for two mode, all vs landmark gene
            if mode == 'all':
                gene_expr_df = lnsGdat_df.groupby('pr_gene_symbol').mean()
                gene_expr_df.drop('-666', inplace=True)
            elif mode == 'lmk':
                directProbe_list = getDirectProbe()
                lnsGdat_df=lnsGdat_df.loc[directProbe_list]
                gene_expr_df = lnsGdat_df.groupby('pr_gene_symbol').mean()

            gene_expr_T_df=gene_expr_df.T
            lnsHead_T_df=lnsHead_df.T
            gene_expr_T_df = pd.concat([lnsHead_T_df,gene_expr_T_df],axis=1)

            ave_gene_expr_df=gene_expr_T_df.groupby(['sm_lincs_id','sm_dose']).mean()

            fold_change_df=ave_gene_expr_df-ave_gene_expr_df.loc[('DMSO','-666')]
            fold_change_df.drop(('DMSO','-666'),axis=0,inplace=True)
            all_fold_change_df=pd.concat([all_fold_change_df,fold_change_df])


        all_fold_change_df.head()

        all_fold_change_df.reset_index(inplace=True)
        all_mean_fold_change_df=all_fold_change_df.groupby(['sm_lincs_id','sm_dose']).mean() 
        
        all_mean_fold_change_df.reset_index(inplace=True)
        
        all_mean_fold_change_df['sm_info']=all_mean_fold_change_df['sm_lincs_id']+'|'+all_mean_fold_change_df['sm_dose']
        all_mean_fold_change_df.drop(['sm_lincs_id','sm_dose'], axis=1, inplace=True)
        
        all_mean_fold_change_df.set_index('sm_info',inplace=True)
        all_mean_fold_change_df.index.name=None
        
        all_mean_fold_change_df.to_csv('result/foldchange/FC_{}_{}.txt'.format(cell,mode),sep='\t')

A375 all


A1CF       A2M    A4GALT     A4GNT      AAAS  \
sm_lincs_id   sm_dose                                                     
BRD-K04833372 0.04    -0.837305 -0.739831 -0.269297 -0.069086 -0.278067   
              0.12    -0.808990  0.027188 -0.305893  0.069430  0.136501   
              0.37    -0.461540  0.124952 -0.059468 -0.041491 -0.255000   
              1.11     0.330895  0.910159  0.221741  0.085517  0.119587   
              10.0    -0.282353 -1.015009  0.065544 -0.250423  0.244980   

                           AACS     AADAC     AAGAB      AAK1      AAMP  \
sm_lincs_id   sm_dose                                                     
BRD-K04833372 0.04    -0.007170  0.338592 -0.270240 -0.061602 -0.245327   
              0.12    -0.039355  0.301167 -0.307061 -0.188495 -0.282346   
              0.37    -0.259230  0.189841 -0.020164 -0.157620  0.016043   
              1.11    -0.233348  0.307213 -0.179372 -0.142020 -0.566891   
              10.0     0.389624  0.006041 -0.171374  0.137772 -0.178387   

                         ...      ZSCAN5A    ZSWIM1      ZW10    ZWILCH  \
sm_lincs_id   sm_dose    ...                                              
BRD-K04833372 0.04       ...     0.259786 -0.020504  0.163759  0.731749   
              0.12       ...     0.209613  0.047348 -0.495758  0.305532   
              0.37       ...    -0.080204 -0.108344  0.182922 -0.022903   
              1.11       ...     0.078868 -0.187481  0.115588 -0.482929   
              10.0       ...     0.138070 -0.075207  0.244417 -0.033929   

                          ZWINT      ZXDB      ZXDC       ZYX     ZZEF1  \
sm_lincs_id   sm_dose                                                     
BRD-K04833372 0.04    -0.496773  0.008175  0.284358 -0.268105 -0.050178   
              0.12    -0.799974  0.166970  0.114829  0.620840 -0.035640   
              0.37     0.255952  0.014974  0.115369 -0.443305 -0.080870   
              1.11    -0.096745  0.110610  0.131025 -0.687776 -0.028597   
              10.0    -0.172665 -0.145431  0.115369  0.154892  0.021612   

                           ZZZ3  
sm_lincs_id   sm_dose            
BRD-K04833372 0.04    -0.432007  
              0.12    -0.610696  
              0.37    -0.125400  
              1.11     0.661025  
              10.0     0.308712  

[5 rows x 12716 columns]

A375 lmk


AARS     ABCB6     ABCC5     ABCF1     ABCF3  \
sm_lincs_id   sm_dose                                                     
BRD-K04833372 0.04     0.335670  0.156553 -0.301464 -0.622617 -0.226277   
              0.12     0.212100  0.242715 -0.066414 -0.194529 -0.111643   
              0.37    -0.219478  0.650706 -0.169753 -0.135502  0.037552   
              1.11    -0.479038 -0.395747 -0.193719 -0.223294 -0.125591   
              10.0    -0.072778 -0.271860 -0.162908 -0.434454  0.583347   

                          ABHD4     ABHD6      ABL1     ACAA1     ACAT2  \
sm_lincs_id   sm_dose                                                     
BRD-K04833372 0.04    -0.225872 -0.061374 -0.249663 -0.014512  0.307341   
              0.12     0.036896 -0.106638 -0.266712 -0.115655  0.111765   
              0.37     0.120058  0.275317 -0.039812 -0.119360  0.112950   
              1.11     0.025786  0.185923  0.017991 -0.018659 -0.174263   
              10.0     0.618144 -0.163277  2.781310  0.112740 -0.287876   

                         ...        ZMIZ1     ZMYM2    ZNF131    ZNF274  \
sm_lincs_id   sm_dose    ...                                              
BRD-K04833372 0.04       ...    -0.273025 -0.013235 -0.109831  0.153643   
              0.12       ...     0.265932 -0.058190 -0.255944  0.275861   
              0.37       ...     0.391828 -0.056741  1.202515 -0.201462   
              1.11       ...    -0.121425  0.043444  0.183654  0.278556   
              10.0       ...     0.283598  0.456111  0.175636 -0.319596   

                         ZNF318    ZNF395    ZNF451    ZNF586    ZNF589  \
sm_lincs_id   sm_dose                                                     
BRD-K04833372 0.04     0.343497 -1.162512  0.013359  4.175951  0.035304   
              0.12     0.356928 -1.568806 -0.022548  0.306611  0.143046   
              0.37     0.271366 -0.133568 -0.226281 -0.304889  0.039448   
              1.11     0.198980  0.328748  0.264217 -0.313326 -0.223727   
              10.0    -0.048198 -1.247369  0.002688 -0.105616 -0.050611   

                           ZW10  
sm_lincs_id   sm_dose            
BRD-K04833372 0.04     0.163759  
              0.12    -0.495758  
              0.37     0.182922  
              1.11     0.115588  
              10.0     0.244417  

[5 rows x 978 columns]

A549 all


...

AARS     ABCB6     ABCC5     ABCF1     ABCF3  \
sm_lincs_id   sm_dose                                                     
BRD-A60245366 0.04    -0.313530 -0.470153 -0.360944 -0.112322  0.212197   
              0.12    -0.152601 -0.441635 -0.667222  0.013520  0.314292   
              0.37    -0.567753 -1.367412 -0.311118 -0.024431  0.377855   
              1.11    -1.425345 -1.809550 -1.114391  0.069575  0.295114   
              10.0    -0.910770  0.088537 -0.516584 -0.705452 -2.591860   

                          ABHD4     ABHD6      ABL1     ACAA1     ACAT2  \
sm_lincs_id   sm_dose                                                     
BRD-A60245366 0.04     0.625218  0.107016  0.350577 -0.102898 -0.024875   
              0.12    -0.022811  0.028827 -0.004803  1.188815 -0.226996   
              0.37     0.311778  0.221584  0.198505  0.191551  0.305403   
              1.11    -0.452110  0.139840 -0.441889 -1.085931 -4.570710   
              10.0    -1.189552  0.210022 -0.380609 -0.859306  0.756564   

                         ...        ZMIZ1     ZMYM2    ZNF131    ZNF274  \
sm_lincs_id   sm_dose    ...                                              
BRD-A60245366 0.04       ...    -0.261397  0.087661 -0.022270 -0.128998   
              0.12       ...     0.637485  0.113465 -0.182551  0.020295   
              0.37       ...     0.135091 -0.129823  0.259075  0.326477   
              1.11       ...    -0.821449 -0.623692 -0.608620  0.879806   
              10.0       ...    -0.695577  0.243605 -0.309657  0.413942   

                         ZNF318    ZNF395    ZNF451    ZNF586    ZNF589  \
sm_lincs_id   sm_dose                                                     
BRD-A60245366 0.04     0.028023  0.006498 -0.160294 -0.124636  2.170434   
              0.12     0.233971 -0.312806 -0.186679  0.129009 -0.567757   
              0.37    -0.599584  0.253323 -0.353193 -0.605714  2.684588   
              1.11    -1.405788  0.065966 -1.417321  6.772274 -0.812533   
              10.0    -0.628520  1.008790 -0.320020 -1.723181 -0.301943   

                           ZW10  
sm_lincs_id   sm_dose            
BRD-A60245366 0.04     0.022351  
              0.12     0.505756  
              0.37     0.620137  
              1.11    -0.155428  
              10.0    -1.510981  

[5 rows x 978 columns]

PC3 all


A1CF       A2M    A4GALT     A4GNT      AAAS  \
sm_lincs_id   sm_dose                                                     
BRD-K25340465 0.04     0.461439  1.850775  0.151329  0.069580  0.107122   
              0.12     0.033559  3.095391  0.144249  0.151974  0.239335   
              0.37    -0.265819 -0.050564 -0.022196  0.003009  0.225332   
              1.11     0.882813  1.436298 -0.323317  0.318653 -0.241289   
              10.0     0.400028  3.310810 -0.047404  0.078524 -0.405224   

                           AACS     AADAC     AAGAB      AAK1      AAMP  \
sm_lincs_id   sm_dose                                                     
BRD-K25340465 0.04     0.364714  0.172423  0.031132 -0.274928  0.059457   
              0.12     0.063906 -0.379318 -0.007224  0.105715 -0.119503   
              0.37    -0.086956  0.090773  0.107248 -0.008025  0.172563   
              1.11     0.305659 -0.607335  0.025073 -0.074722  0.082186   
              10.0    -0.345679  0.320801  0.864485 -0.085725 -0.122747   

                         ...      ZSCAN5A    ZSWIM1      ZW10    ZWILCH  \
sm_lincs_id   sm_dose    ...                                              
BRD-K25340465 0.04       ...     0.168872  0.049854  0.184526 -0.166836   
              0.12       ...     0.188582  0.280928 -0.080102 -0.043061   
              0.37       ...    -0.059795  0.084512 -0.039197  0.273347   
              1.11       ...     0.123859  0.069438 -2.075090 -0.048153   
              10.0       ...    -0.649167  0.125674 -2.007493  0.292189   

                          ZWINT      ZXDB      ZXDC       ZYX     ZZEF1  \
sm_lincs_id   sm_dose                                                     
BRD-K25340465 0.04     0.192024 -0.029062  0.188056  0.023430  0.139041   
              0.12    -0.186176 -0.015820 -0.045974  0.173257 -0.036571   
              0.37     0.272545  0.049868  0.319688  0.394473  0.120163   
              1.11     0.363952  0.103266 -0.093193  0.413753 -0.000864   
              10.0    -1.429476 -0.099965  0.465733  0.237444  0.141888   

                           ZZZ3  
sm_lincs_id   sm_dose            
BRD-K25340465 0.04     0.495476  
              0.12    -0.105183  
              0.37     0.569086  
              1.11    -0.005503  
              10.0    -0.527116  

[5 rows x 12716 columns]

PC3 lmk


AARS     ABCB6     ABCC5     ABCF1     ABCF3  \
sm_lincs_id   sm_dose                                                     
BRD-K25340465 0.04     0.044565  0.391383  0.012661  0.012274  0.062659   
              0.12     0.165076  0.514117 -0.618950 -0.034233 -0.036083   
              0.37     0.077618 -1.145284 -0.772949 -0.119469 -0.077942   
              1.11     0.059301  0.530989 -0.351321  0.158792 -0.200765   
              10.0     0.597652  0.497000 -0.512178 -0.167241 -0.629958   

                          ABHD4     ABHD6      ABL1     ACAA1     ACAT2  \
sm_lincs_id   sm_dose                                                     
BRD-K25340465 0.04     0.217811 -0.623904 -0.076997  1.454345  0.583667   
              0.12     0.023112 -0.655570 -0.254773 -0.118114  1.019148   
              0.37    -0.342342 -0.706365  0.055978 -0.240198 -0.102754   
              1.11     0.029664 -0.634954 -0.061605 -0.400040  0.408926   
              10.0     0.947347 -0.603893 -0.265349 -0.785119  0.725314   

                         ...        ZMIZ1     ZMYM2    ZNF131    ZNF274  \
sm_lincs_id   sm_dose    ...                                              
BRD-K25340465 0.04       ...    -0.179268  0.174266  1.413863  0.408709   
              0.12       ...    -0.856010  0.088284  0.619343 -0.526763   
              0.37       ...    -0.181123  0.107630 -0.472629  0.341099   
              1.11       ...     0.033636  0.234818 -0.652423 -0.680633   
              10.0       ...     0.221901  0.091703  0.654248  0.152598   

                         ZNF318    ZNF395    ZNF451    ZNF586    ZNF589  \
sm_lincs_id   sm_dose                                                     
BRD-K25340465 0.04     0.471108  0.306788  0.295623 -0.755810 -0.530166   
              0.12     0.454117  0.460533  0.019024 -1.382964  1.408309   
              0.37    -0.302462  0.110531  0.049385 -1.044311 -0.700911   
              1.11     0.939694  0.228148  0.070486 -0.753083  1.305125   
              10.0    -0.189357  0.377277  0.246802 -0.753600  1.423328   

                           ZW10  
sm_lincs_id   sm_dose            
BRD-K25340465 0.04     0.184526  
              0.12    -0.080102  
              0.37    -0.039197  
              1.11    -2.075090  
              10.0    -2.007493  

[5 rows x 978 columns]